# Welcom to EdgeLab for Google Colab Training Example 🔥 

<a href="https://colab.research.google.com/github/Seeed-Studio/EdgeLab/blob/main/notebooks/Google-Colab-PFLD-Grove-Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> **[🚀🚀🚀 One-Click to Deploy to Google Colab 🚀🚀🚀](https://colab.research.google.com/github/Seeed-Studio/EdgeLab/blob/main/notebooks/Google-Colab-PFLD-Grove-Example.ipynb)**

In this tutorial, we will demonstrate how to use [EdgeLab](https://github.com/Seeed-Studio/EdgeLab) and Colab to train a PFLD model for meter reading ⏱️.

Table of contents:

- [Setup EdgeLab](#setup-edgelab)
- [Download Custom Meter Datasets](#download-custom-meter-datasets)
- [Train PFLD Model](#train-pfld-model)
- [Optimize PFLD Model and Export](#optimize-pfld-model-and-export)
- [Deploy to Grove Vision AI](#deploy-to-grove-vision-ai)
- [Thanks for Trying out EdgeLab 🎉](#thanks-for-trying-out-edgelab-🎉)

For more details of PFLD model or application scenarios, please refer to [EdgeLab Documentation](https://seeed-studio.github.io/EdgeLab/tutorials/training/pfld).

**Tips: Since model training consumes a lot of computational resources, we recommend that you run this notebook on [Colab's GPU instances](https://research.google.com/colaboratory/faq.html#gpu-availability).**

## Setup EdgeLab

**Step 0:** Clone the EdgeLab source code from [EdgeLab GitHub repository](https://github.com/Seeed-Studio/EdgeLab), and enter the EdgeLab project directory.

In [ ]:
!git clone https://github.com/Seeed-Studio/EdgeLab.git  # currently we're using experimental 2.0 version branch

In [ ]:
%cd /content/EdgeLab

**Step 1:** Use `python3.8` as default python in colab, then install corresponding version python and pip package.

In [ ]:
%env PYTHON_EXEC=python3.8

In [ ]:
!sudo apt-get update
!sudo apt-get install ${PYTHON_EXEC}-dev python3-pip -y

**Step 2:** Use `scripts/setup_colab.sh` to automatically setup EdgeLab dependencies.

In [ ]:
!chmod +x scripts/setup_colab.sh
!scripts/setup_colab.sh $(which ${PYTHON_EXEC})

## Download Custom Meter Datasets

First, download and unpack our custom meter datasets.

For more available datasets, please refer to [EdgeLab Documentation - Datasets](https://seeed-studio.github.io/EdgeLab/tutorials/datasets).

In [ ]:
!wget https://files.seeedstudio.com/wiki/Edgelab/meter.zip -P datasets/
!unzip datasets/meter.zip -d datasets/

## Train PFLD Model

### Examine the PFLD config file

The config we use for train is located at `configs/pfld/pfld_mv2n_112.py`.

For more config details, please refer to [EdgeLab Documentation - Config](https://seeed-studio.github.io/EdgeLab/tutorials/config).

In [ ]:
!cat configs/pfld/pfld_mbv2n_112.py

### Download Pre-train weights

In [ ]:
!wget -P pre-train/ https://github.com/Seeed-Studio/EdgeLab/releases/download/model_zoo/pfld_mbv2n_112.pth 

### Train the model using default config

Then, train PFLD model using default config, note here we need to override the config by 2 parameters:

- `data_root` - the datasets path, which located at path `datasets/meter`

- `epochs`- the train epochs, we use `50` to reduce the training time

- `load_from` the pre-train weights, make training faster

For more training details, please refer to [EdgeLab Documentation - Train PFLD Model](https://seeed-studio.github.io/EdgeLab/tutorials/training/pfld).

In [ ]:
!${PYTHON_EXEC} tools/train.py \
  configs/pfld/pfld_mbv2n_112.py \
  --cfg-options \
    epochs=50 \
    num_classes=1 \
    data_root='datasets/meter' \
    load_from='pre-train/pfld_mbv2n_112.pth'

The output directory in the training process would be look like this:

In [ ]:
!ls -alh work_dirs/pfld_mbv2n_112

We could inspect the latest model weight path by this command:

In [ ]:
!echo "Latest model weights path: $(cat work_dirs/pfld_mbv2n_112/last_checkpoint)"

### Validate the Model

We recommend you to validate the model with the latest weight after training using the `tool/test.py` script.

In [ ]:
!${PYTHON_EXEC} tools/inference.py \
    configs/pfld/pfld_mbv2n_112.py \
    "$(cat work_dirs/pfld_mbv2n_112/last_checkpoint)" \
    --dump work_dirs/pfld_mbv2n_112/last_checkpoint.pkl \
    --cfg-options \
        data_root='datasets/meter'

In [ ]:
import pickle

with open('work_dirs/pfld_mbv2n_112/last_checkpoint.pkl', 'rb') as f:
    data = pickle.load(f)
    for k in data.keys():
      print(f'{k}: {data[k]:.5f}')

## Optimize PFLD Model and Export

### Export PFLD PyTorch model to TFLite model

Here we're using `tools/torch2tflite.py` to convert and export the latest weights to a TFLite model at a INT8 precision for inference.

In [ ]:
!${PYTHON_EXEC} tools/export.py \
  configs/pfld/pfld_mbv2n_112.py \
  $(cat work_dirs/pfld_mbv2n_112/last_checkpoint) \
  tflite \
  --cfg-options \
    data_root='datasets/meter'

## Deploy to Grove Vision AI

This example is a tutorial for deploying the models from [EdgeLab](https://github.com/Seeed-Studio/Edgelab/) to Grove - Vision AI module, based on the [Synopsys GUN Toolchain](https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain) and [Tensorflow Lite Micro](https://github.com/tensorflow/tflite-micro) implementations.

For more details, please refer to [EdgeLab Documentation - Example - Grove - Deploy](https://seeed-studio.github.io/EdgeLab/examples/grove/deploy#compile-and-deploy).

### Download and Setup Build Tool Chain

In [ ]:
!wget https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain/releases/download/arc-2020.09-release/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz -P ~/

In [ ]:
!tar -zxf ~/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz --directory ~/

In [ ]:
SYSTEM_PATH = %env PATH
SYSTEM_PATH = SYSTEM_PATH.replace('"', '')
SYSTEM_HOME = %env HOME
SYSTEM_HOME = SYSTEM_HOME.replace('"', '')
SYSTEM_PATH = f'{SYSTEM_HOME}/arc_gnu_2020.09_prebuilt_elf32_le_linux_install/bin:{SYSTEM_PATH}'
%set_env PATH={SYSTEM_PATH}

### Clone and Setup Grove Vision AI SDK

In [ ]:
!git clone https://github.com/Seeed-Studio/edgelab-example-vision-ai example/grove

In [ ]:
!cd example/grove

In [ ]:
!${PYTHON_EXEC} -m pip install numpy requests colorama serial pyserial

### Build Grove Vision AI Firmware Img

In [ ]:
!cd example/grove && \
  make HW=grove_vision_ai APP=meter

### Convert Firmware Image to UF2

In [ ]:
!cd example/grove && \
  ${PYTHON_EXEC} tools/ufconv/uf2conv.py \
    -t 0 \
    -c tools/image_gen_cstm/output/output.img \
    -o firmware.uf2

In [ ]:
!echo "The UF2 firmware is located at: $(pwd)/example/grove/firmware.uf2"

In [ ]:
from IPython.display import FileLink

FileLink('example/grove/firmware.uf2')

### Convert Trained Model (TFLite) to UF2

In [ ]:
!cd example/grove && \
  ${PYTHON_EXEC} tools/ufconv/uf2conv.py \
    -t 1 \
    -c "$(cat ../../work_dirs/pfld_mbv2n_112/last_checkpoint | sed -e 's/.pth/_int8.tflite/g')" \
    -o model.uf2

In [ ]:
!echo "The UF2 model is located at: $(pwd)/example/grove/model.uf2"

In [ ]:
FileLink('example/grove/model.uf2')

Please download the UF2 Firmware and UF2 Model and flash them into a Grove Vision AI module. Finally, you can see the real-time meter reading results as shown in the figure below.

![Meter Reader](https://seeed-studio.github.io/EdgeLab/static/grove/images/pfld_meter.gif)

For more information on **how to use these UF2 images**, please refer to [EdgeLab Documentations - Deploy - Grove - Deployment Routines](https://seeed-studio.github.io/EdgeLab/examples/grove/deploy#deployment-routines).

## Thanks for Trying Out EdgeLab 🎉

Congratulations, you have completed this tutorial. If you are interested in more application scenarios or our projects, please feel free to give [EdgeLab](https://github.com/Seeed-Studio/EdgeLab) a star ✨ on GitHub.

If you have any questions about this tutorial, please also feel free to [submit an issue](https://github.com/Seeed-Studio/EdgeLab/issues).